He visto que dedicais demasiadas horas a las practicas. Recuerdo la regla intentais pasar un maximo de 2h.
A partir de ahora, cambiamos un poco los criterios de avaluacion, hasta ahora yo miraba si funcionava, si habiais hecho el trabajo. Pero a partir de ahora intentaremos sobre todo programar "bien", es decir seguir las siguientes reglas:
    - comentarios: codigo comentado para que se entienda, los comentarios tienen que ser presentes pero tambien ser concisos para evitar sobre caregar inutilmente el codigo
    - nombre de variables explicito: elegir con cuidado los nombres de variable para que el codigo sea facil de entender
    - Optimizacion: Intentaremos no hacer trabajar el ordenador inutilmente, es decir, utilizar numpy si es possible en ves de las loops, no hacer loops inutiles, no escribir lineas inutiles (por ejemplo incrementar el contador dentro de un for).
    - simple es mejor que complicado: a veces mejor buscar un poco mas una manera sencilla y elegante de hacer las cosas es mejor que escribir toneladas de for, de if, y de while
    - escribir funciones: cuando teneis que repetir una operacion varias veces, mejor hacer una funcion. No olvidar escribir la documentacion de la funciones
    - Hardcoded prohibido: poner los numeros en variables, nunca hacer un print y copiar el valora mano. (excepto por los nombre de varible de los fichieros netcdf y para definir el rango de una figura).
    - Control de error: Es mejor cuando se escriba una funcion pensar a comprovar que los arguementos esten bien, por ejemplo sus dimensiones, comprovar que los fichieros existen...

Sigo poniendo 2 niveles de dificuldad y un tiempo estimado

## Que controla la precipitaciones en hivierno?

### Part 1 - Tendencia (facil - 7pt)

1) Leer el fichiero de precipitacion de GPCP sobre Europa (https://www.esrl.noaa.gov/psd/data/gridded/data.gpcp.html). Utilizando el servidor Opendab o descargando el fichiero. Convertir los valores en mm/day. (tiempo max: 25min - 1 pt)

In [3]:
#import netcdf library
from netCDF4 import Dataset as nc 
#import date conversor netcdf
from netCDF4 import num2date, date2num 
from datetime import date, timedelta, datetime
import numpy as np
import os
from scipy import stats
import sys

In [4]:
#definimos una funcion que coge en argumento un fichiero unas latitude y longitude 
#y devuelve los datos correspondientes
def readregion(file, varname, lon_bnds, lat_bnds):
    """funcion que lee los datos 3D en una region
    filename: path de un fichiero netcdf
    varname: nombre de la variable
    lon_bnds: array, list o tuple que contiene los limites de la caja a selecionar
    lat_bnds: array, list o tuple que contiene los limites de la caja a selecionar
    return: var, time, lon, lat
    """
    
    #empezamos por leer el fichiero ncep de t2m global
    try :
        file = nc(filename, "r") # open the netcdf file 
    except:
        print("el fichiero "+filename+" no existe.")
        sys.exit(1)
    #leer time axis
    if "time" in file.variables.keys():
        timevar = file.variables["time"]
    else:
        print("no variable time in the file")
        sys.exit(1)
        
    dates = num2date(timevar[:], units=timevar.units) 
    #leer longitude
    if "lon" in file.variables.keys():    
        lon = file.variables["lon"][:]
    else:
        print("no variable lon in the file")
        sys.exit(1)
    #leer latitude
    if "lat" in file.variables.keys():    
        lat = file.variables["lat"][:]
    else:
        print("no variable lat in the file")
        sys.exit(1)
    #check the existence and the dimension of varname
    if not(varname in file.variables.keys()):
        print("no variable "+varname+" in the file")
        sys.exit(1)
    elif len(file.variables[varname].shape)!=3:
        print("This function is done to read varable of size 3")
        sys.exit(1)
        
    #convitimos las listas en numpy array 
    lon_bnds = np.array(lon_bnds)
    lat_bnds = np.array(lat_bnds)
    # if longitude is written in negative convert it in value from 0 to 360 
    #(if it is consistent with the longitude of the file we want to read)
    #print(lon)
    lon_bnds = (lon_bnds + 360)%360
    #print(lon_bnds)
    #latitude correspondientes a la caja
    latbox = np.where((lat > lat_bnds[0])&(lat<lat_bnds[1]))[0]
    lat = lat[latbox]
        
    if lon_bnds[0]>lon_bnds[1]:
        #print("I am here")
        #calculamos los limites de la caja al oeste de greenwitch
        lonbox1 = np.where((lon >= lon_bnds[0]))[0]
        #calculamos los limites de la caja al este de greenwitch
        lonbox2 = np.where((lon <= lon_bnds[1]))[0]
        #extraemos los datos del oeste de greenwitch
        var1 = file.variables[varname][:,latbox,lonbox1]
        #extraemos los datos del este de greenwitch
        var2 = file.variables[varname][:,latbox,lonbox2]
        #juntamos los dos cajas
        var = np.concatenate((var1, var2), axis = 2)
        lon = np.concatenate((lon[lonbox1], lon[lonbox2]), axis = 0)
    else:
        lonbox = np.where((lon >= lon_bnds[0]) & (lon <= lon_bnds[1]))[0]
        lon = lon[lonbox]
        var = file.variables[varname][np.where(dates<datetime(2019,1,1)),latbox,lonbox]
    return(var, dates, lon, lat)

In [5]:
lon_bnds = [-20, 40]
lat_bnds = [30, 80]
filename = "https://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/gpcp/precip.mon.mean.nc"
precip, dates, lon, lat = readregion(filename, "precip", lon_bnds, lat_bnds)

In [20]:
dates

array([datetime.datetime(1979, 1, 1, 0, 0),
       datetime.datetime(1979, 2, 1, 0, 0),
       datetime.datetime(1979, 3, 1, 0, 0),
       datetime.datetime(1979, 4, 1, 0, 0),
       datetime.datetime(1979, 5, 1, 0, 0),
       datetime.datetime(1979, 6, 1, 0, 0),
       datetime.datetime(1979, 7, 1, 0, 0),
       datetime.datetime(1979, 8, 1, 0, 0),
       datetime.datetime(1979, 9, 1, 0, 0),
       datetime.datetime(1979, 10, 1, 0, 0),
       datetime.datetime(1979, 11, 1, 0, 0),
       datetime.datetime(1979, 12, 1, 0, 0),
       datetime.datetime(1980, 1, 1, 0, 0),
       datetime.datetime(1980, 2, 1, 0, 0),
       datetime.datetime(1980, 3, 1, 0, 0),
       datetime.datetime(1980, 4, 1, 0, 0),
       datetime.datetime(1980, 5, 1, 0, 0),
       datetime.datetime(1980, 6, 1, 0, 0),
       datetime.datetime(1980, 7, 1, 0, 0),
       datetime.datetime(1980, 8, 1, 0, 0),
       datetime.datetime(1980, 9, 1, 0, 0),
       datetime.datetime(1980, 10, 1, 0, 0),
       datetime.datetime(198

2) Escribir una funcion que calcula las anomalias y calcular las anomalias de precipitaciones (tiempo max: 30min - 2pt)

In [ ]:
def compute_anom

3) Calcular la media estacional de la precipitaciones en verano (June Julio Agosto) y hivierno (Decembre Enero Febrero, cuidado teneis que juntar los meses seguidos, por ejemplo Decembre 1993 con Enero 1994 y Febrero 1994) (30min - 2pt)

3) Calcular la tendencia de la precipitacion en verano y hiverno (30 min - 2 pt)

4) Hacer los mapas de la tendencia de precipitacion en hiverno y verano, incluendo la pvalue (40min - 3pt)

### Part 2 - NAO mes dificil (2pt)

3) Leer los datos de geopotential para el calculo de la NAO https://www.youtube.com/watch?v=KOYJG7j4Iy8 - Norte punto alrededor de -25ºW, 65ºN y cara sur alrededor de -25ºW y 35ºN

In [ ]:

filename = "https://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/ncep.reanalysis.derived/pressure/hgt.mon.mean.nc"
# ahora podemos buscar los dos puntos que no interessan
Northpoint = (-22, 64)
Southpoint = (-25, 38)

#buscar el level mas cercano a 500
indlev500 = (np.abs(lev-500)).argmin()

#encontrar la position de la latitude y longitude correspondiente a estos punto
latNorth = (np.abs(lat-Northpoint[1])).argmin()
lonNorth = (np.abs(lon-Northpoint[0])).argmin()

latSouth = (np.abs(lat-Southpoint[1])).argmin()
lonSouth = (np.abs(lon-Southpoint[0])).argmin()

geoNorth = fileTmax.variables["hgt"][:,indlev500,latNorth,lonNorth]
geoSouth = fileTmax.variables["hgt"][:,indlev500,latNorth,lonNorth]

4) Calcular las anomalies de los dos puntos de la NAO. 


5) Normalizar las anomalies en los dos punto (es decir dividir por la deviation estandard) y hacer la differencia entre las dos series de anomalies - Asi teneis el indice de la NAO. 

6) Calcular la media estacional de la NAO

7) Calcular la regression entre los el indice de NAO y los datos de temperatura en hivierno 

8) Hacer el mapa (incluyendo la pvalue), escribir un pequeño comentario de los que podeis deducir de esta figura. (tiempo max: 20min, 1pt)

### Part 2 - Otros factores (ENSO - Tendencia - 2pt)

Calcular la regresion con ENSO de la precipitacion hiviernal en Europa. Hacer los mapas correspondientes.